## Looking for the right neighborhood to move in Chicago?

In [21]:
%%html
<img src="Pictures/Intro.png", width = 1000, height = 600>

#### Chicago is considered one of the most popular cities in the US. This case study targets to support „young families“ willing to move to Chicago by finding a suitable neighborhood. 

In [20]:
%%html
<img src="Pictures/family.png", width = 300, height = 300>

#### Target Audience: Young families (professionals) looking for a safe environment to grow up their children, close to schools, but at the same time having the flexibility to chose places closed to their interests (e.g. sport, restaurants etc.)

In [19]:
%%html
<img src="Pictures/criteria.png", width = 600, height = 300>

In [4]:
import pandas as pd

import numpy as np

import requests

### 1. Data Sources Nr.1: Chicago Crime data from 2001 to present
This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. Data is extracted from the Chicago Police Department's CLEAR (Citizen Law Enforcement Analysis and Reporting) system.

In [7]:
chicago_crime_df = pd.read_csv ('Crimes_-_2001_to_present.csv')
chicago_crime_df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,12026980,JD205512,03/26/2020 12:00:00 PM,004XX W PERSHING RD,1206,DECEPTIVE PRACTICE,"THEFT BY LESSEE, MOTOR VEHICLE",COMMERCIAL / BUSINESS OFFICE,False,False,...,11.0,61.0,11,NaN,NaN,2020.0,04/10/2020 03:47:54 PM,NaN,NaN,NaN
1,12022939,JD201040,04/03/2020 11:40:00 AM,017XX W 47TH ST,1310,CRIMINAL DAMAGE,TO PROPERTY,BANK,False,False,...,15.0,61.0,14,1165336.0,1873531.0,2020.0,04/10/2020 03:47:54 PM,41.808564,-87.669106,"(41.808564273, -87.669106152)"
2,12024662,JD202937,04/03/2020 10:00:00 PM,031XX S WESTERN AVE,0460,BATTERY,SIMPLE,STREET,False,False,...,25.0,31.0,08B,1160938.0,1884070.0,2020.0,04/10/2020 03:47:54 PM,41.837577,-87.684945,"(41.837576666, -87.684945388)"
3,12022867,JD200881,04/03/2020 07:45:00 AM,024XX W POPE JOHN PAUL II DR,141A,WEAPONS VIOLATION,UNLAWFUL USE - HANDGUN,APARTMENT,False,False,...,15.0,58.0,15,1160631.0,1876084.0,2020.0,04/10/2020 03:47:54 PM,41.815669,-87.686293,"(41.815668509, -87.686292611)"
4,12023185,JD201388,04/03/2020 06:25:00 PM,043XX S TALMAN AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,15.0,58.0,14,1159423.0,1875798.0,2020.0,04/10/2020 03:47:54 PM,41.814909,-87.690732,"(41.814908564, -87.690731646)"


In [26]:
chicago_crime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350592 entries, 0 to 350591
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    350592 non-null  int64  
 1   Case Number           350592 non-null  object 
 2   Date                  350592 non-null  object 
 3   Block                 350592 non-null  object 
 4   IUCR                  350592 non-null  object 
 5   Primary Type          350592 non-null  object 
 6   Description           350592 non-null  object 
 7   Location Description  350471 non-null  object 
 8   Arrest                350592 non-null  bool   
 9   Domestic              350592 non-null  bool   
 10  Beat                  350592 non-null  int64  
 11  District              350592 non-null  int64  
 12  Ward                  319660 non-null  float64
 13  Community Area        319629 non-null  float64
 14  FBI Code              350592 non-null  object 
 15  

In [22]:
chicago_crime_df.shape

(350592, 22)

### 1a. Data Preprocessing
#### Dropping empty rows
As shwon with the info(), there are some rows with missing values. Especially for Latitute and Longitue we need all the available values as well as for the community area identifier (identifier to join with our second dataset). We remove these rows using the following line.

#### 1.a) Get some insights about the data       

In [56]:
view = chicago_crime_df.groupby('Primary Type').count()[['Case Number']]
view.head()

,Case Number
Primary Type,
ARSON,138
ASSAULT,3422
BATTERY,8784
BURGLARY,2342
CONCEALED CARRY LICENSE VIOLATION,13


In [60]:
view.sort_values(by ='Case Number', ascending = False)

,Case Number
Primary Type,
BATTERY,8784
THEFT,8701
CRIMINAL DAMAGE,5427
ASSAULT,3422
OTHER OFFENSE,2903
NARCOTICS,2662
DECEPTIVE PRACTICE,2488
BURGLARY,2342
ROBBERY,2036


In [70]:
chicago_crime_df['Community Area'].nunique()

14

In [29]:
public_schools_df= pd.read_csv ('Chicago_Public_Schools.csv')
public_schools_df.head()

,School ID,Name of School,"Elementary, Middle, or High School",Street Address,City,State,ZIP Code,Phone Number,Link,Network Manager,...,RCDTS Code,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,Community Area Number,Community Area Name,Ward,Police District,Location
0,609966,Charles G Hammond Elementary School,ES,2819 W 21st Pl,Chicago,IL,60623,(773) 535-4580,http://schoolreports.cps.edu/SchoolProgressRep...,Pilsen-Little Village Elementary Network,...,150000000000000,1157809.015,1889554.623,41.852691,-87.696278,30,SOUTH LAWNDALE,12,10,"(41.85269133, -87.69627777)"
1,610539,Marvin Camras Elementary School,ES,3000 N Mango Ave,Chicago,IL,60634,(773) 534-2960,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,150000000000000,1137482.296,1919394.900,41.934966,-87.770165,19,BELMONT CRAGIN,30,25,"(41.93496641, -87.77016525)"
2,609852,Eliza Chappell Elementary School,ES,2135 W Foster Ave,Chicago,IL,60625,(773) 534-2390,http://schoolreports.cps.edu/SchoolProgressRep...,Ravenswood-Ridge Elementary Network,...,150000000000000,1161016.902,1934466.545,41.975867,-87.683254,4,LINCOLN SQUARE,47,20,"(41.975867, -87.68325438)"
3,609835,Daniel R Cameron Elementary School,ES,1234 N Monticello Ave,Chicago,IL,60651,(773) 534-4290,http://schoolreports.cps.edu/SchoolProgressRep...,Garfield-Humboldt Elementary Network,...,150000000000000,1151767.546,1908130.068,41.903785,-87.717963,23,HUMBOLDT PARK,26,25,"(41.90378521, -87.71796315)"
4,610521,Sir Miles Davis Magnet Elementary Academy,ES,6730 S Paulina St,Chicago,IL,60636,(773) 535-9120,http://schoolreports.cps.edu/SchoolProgressRep...,Englewood-Gresham Elementary Network,...,150000000000000,1166133.893,1859929.018,41.771222,-87.666567,67,WEST ENGLEWOOD,15,7,"(41.77122181, -87.66656657)"


In [ ]:
# print pd.merge(chicago_crime_df, public_schools_df, on='Customer_id', how='left')

In [32]:
view = public_schools_df.groupby('Community Area Name').count()[['School ID']]
view


,School ID
Community Area Name,
ALBANY PARK,8
ARCHER HEIGHTS,2
ARMOUR SQUARE,3
ASHBURN,8
AUBURN GRESHAM,10
...,...
WEST LAWN,4
WEST PULLMAN,10
WEST RIDGE,9


In [33]:
view2 = public_Schools_df.groupby('Community Area Number').count()[['School ID']]
view2


,School ID
Community Area Number,
1,6
2,9
3,7
4,5
5,7
...,...
73,9
74,4
75,5


In [67]:
zip_codes_df.shape

(566, 79)